# 極性アミノ酸用

In [3]:
from Bio.PDB import PDBParser, MMCIFParser, Selection
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio.PDB.DSSP import DSSP
from scipy.spatial import KDTree
from Bio.SeqUtils import seq1
from pprint import pprint
import networkx as nx
import numpy as np
import pandas as pd
import os

class ResiduePatch():
    """
    Parse pisite files to dict

    Attributes
    ----------
    ids : list
        residue ids
    file: str
        pdb file
    dssp: dict
        dssp dict
    dssp_dict_keys: dict
        dssp dict keys
    """

    def __init__(self, ids, dssp, dssp_dict_keys):
        self.ids = ids
        #print(ids)
        self.dssp = dssp
        #print(dssp_dict_keys)
        self.dssp_dict_keys = dssp_dict_keys

    def get_ids(self):
        return self.ids

    def size(self):
        absolute_asa = {"G":104,"M":224,"A":129,"P":159,"V":174,"F":240,"L":201,"I":197,"W":285}
        #return sum(self.dssp[i[-2:]][2] for i in self.ids)#edited
        return sum(self.dssp[i[-2:]][3] * absolute_asa[self.dssp[i[-2:]][1]] for i in self.ids)
    def residues(self):
        #return [self.dssp[i[-2:]][0] for i in self.ids]
        return [self.dssp[i[-2:]][1] for i in self.ids]
    def patch_length(self):
        return len(self.ids)

    def residue_on_surface(self):
        """
        Get all residues accessible from the surface

        Return
        ------
        list
            residue ids
        """
        residue_list = []
        for x in self.dssp_dict_keys:
            if x[1][0] != ' ' or  x[1][2] != ' ':
                continue
            if self.dssp[x][2] > 0:
                residue_list.append(x)
        #print(residue_list)
        return residue_list
        
class ProteinPatch():

    def __init__(self, id, file, residues_in_patch, r=1.25, msms='msms -density 1.5'):
        if file.endswith('.pdb'):
            parser = PDBParser()    
        elif file.endswith('.cif'):
            parser = MMCIFParser()
        structure = parser.get_structure(id, file)
        self.model = structure[0]
        self.r = r
        if not msms.startswith('msms'):
            msms = 'msms ' + msms
        self.msms = msms
        self.residues_in_patch = residues_in_patch
        self.dssp_dict = DSSP(self.model, file)
        
        self.dssp_dict_keys = self.dssp_dict.keys()
        #print(self.dssp_dict[self.dssp_dict_keys[2]][3])#dict[key][3]
        #self.dssp_dict, self.dssp_dict_keys =  dssp_dict_from_pdb_file(file)
        self.G = self.dot_cloud_graph()
        self.G = self.patch_network(self.G)
        self.patches = self.create_patches()

    def dot_cloud_graph(self):
        """
        Create a dotted cloud of the proteins surface and label the dots
        hydrophobic or hydrophilic
        ...

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic or not
        """
        surface_points = get_surface(self.model, MSMS=self.msms)
        residue_list = [r for r in Selection.unfold_entities(self.model, "R") if seq1(r.get_resname()) != 'X']
        center_vectices = [self._sidechain_center(r.get_atoms()) for r in residue_list]

        T = KDTree(center_vectices)
		
        closest_residues = T.query(surface_points, k=1)[1]

        G = nx.Graph()
        for node, coordinates in enumerate(surface_points):
            G.add_node(node)
            G.nodes[node]['selected'] = 0
            closest_residue = residue_list[closest_residues[node]]
            if seq1(closest_residue.get_resname()) in self.residues_in_patch:
                G.nodes[node]['selected'] = 1
            G.nodes[node]['surface_vector_pos'] = coordinates
            G.nodes[node]['closest_residue_id'] = closest_residue.get_full_id()
            G.nodes[node]['closest_residue_aa'] = seq1(closest_residue.get_resname())
        return G

    def patch_network(self, G):
        """
        Create a edges between hydrophobic nodes if they are within r
        hydrophobic or hydrophilic
        ...

        Attributes
        ----------
        G : networkx.Graph
            Graph with nodes labeled as hydrophobic or not

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic and edges between nodes within range r
        """
        node_list = [i for i in G.nodes if G.nodes[i]['selected']]
        x = [G.nodes[i]['surface_vector_pos'] for i in G.nodes if G.nodes[i]['selected']]
        T = KDTree(x)
        pairs = T.query_pairs(self.r)
        G.add_edges_from([(node_list[x[0]],node_list[x[1]]) for x in pairs])
        return G

    def largest_patch(self):
        """
        get largest patch

        Return
        ------
        networkx.Graph
            Graph of the largest patch
        """
        largest_patch = max(self.patches, key=(lambda x: x.size()))
        
        return largest_patch

    def create_patches(self):
        """
        Get the components of the graph

        Return
        ------
        list
            list of Graph components where an item is a patch
        """
        patched_G = self.patch_network(self.G)
        components = [patched_G.subgraph(c) for c in nx.connected_components(patched_G)] #nx.connected_component_subgraphs(patched_G)
        patch_dict = []
        for component in components:
            if len(component.nodes) <= 1:
                continue
            residue_ids_in_patch = list(set([component.nodes[i]['closest_residue_id'] for i in component.nodes]))
            patch_dict.append(ResiduePatch(residue_ids_in_patch, self.dssp_dict, self.dssp_dict_keys))
        #print([i.size() for i in patch_dict])
        return sorted(patch_dict, key=(lambda x: x.size()), reverse=True)

    def _sidechain_center(self, atoms):
        vectors = [atom.get_vector().get_array() for atom in atoms]
        center = np.array(vectors).mean(axis=0)
        return center
def calculate_patches(infile, residues, plot, output_dir):
    """ Calculate patches on a protein surface """

    filename = os.path.basename(infile)
    outfile = filename.split('.')[0] + '.csv'
    pdb_id = filename.split('.')[0].split('_')[0]

    proteinPatches = ProteinPatch(pdb_id,infile,residues)
    patches = proteinPatches.patches
    #print([i.get_ids() for i in patches])
    result_dict = {'patch_rank':[], 'protein_id':[], 'residue_id':[], 'chain':[], 'residue_type':[], 'patch_size':[]}
    for i,patch in enumerate(patches):
        residues_in_patch = patch.get_ids()
        residues = patch.residues()
        #print(residues)
        for j, residue_in_patch in enumerate(residues_in_patch):
            result_dict['residue_id'].append(residue_in_patch[-2:][1][1])
            result_dict['chain'].append(residue_in_patch[-2:][0])
            result_dict['patch_size'].append(patch.size())
            result_dict['patch_rank'].append(i)
            result_dict['residue_type'].append(residues[j])
            result_dict['protein_id'].append(pdb_id)
            

    pd.DataFrame(result_dict).to_csv(os.path.join(output_dir, "patches_" + outfile), index=False)

    if plot:
        outfig = infile.split('.')[0] + '.png'
        proteinPatches.plot_largest_patches(outfig)

In [4]:
file = '../pdb_files/minus1.pdb'
output_dir = '../output/'
residues = ["G","M","A","P","V","F","L","I","W"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

['A', 'G', 'A', 'G', 'I', 'P', 'F', 'A', 'A', 'A', 'M', 'A', 'G', 'A', 'A', 'P', 'F', 'L', 'A', 'A', 'F', 'P', 'A', 'A', 'A', 'A', 'F', 'L', 'G', 'I', 'V', 'G', 'A', 'P']


In [114]:
file = '../pdb_files/plus1.pdb'
output_dir = '../output/'
residues = ["G","M","A","P","V","F","L","I","W"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [120]:
file = '../pdb_files/minus2.pdb'
output_dir = '../output/'
residues = ["G","M","A","P","V","F","L","I","W"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [126]:
file = '../pdb_files/plus2.pdb'
output_dir = '../output/'
residues = ["G","M","A","P","V","F","L","I","W"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [132]:
file = '../pdb_files/plus2_ala_0.pdb'
output_dir = '../output/'
residues = ["G","M","A","P","V","F","L","I","W"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [136]:
file = '../pdb_files/plus2_ala_A.pdb'
output_dir = '../output/'
residues = ["G","M","A","P","V","F","L","I","W"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [141]:
for name in ['sample4','sample5','sample6','sample7','sample8','sample9']:
    file = '../pdb_files/' + name + '.pdb'
    output_dir = '../output/'
    residues = ["G","M","A","P","V","F","L","I","W"]
    calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

# 非極性アミノ酸用

In [146]:
from Bio.PDB import PDBParser, MMCIFParser, Selection
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio.PDB.DSSP import DSSP
from scipy.spatial import KDTree
from Bio.SeqUtils import seq1
from pprint import pprint
import networkx as nx
import numpy as np
import pandas as pd
import os

class ResiduePatch():
    """
    Parse pisite files to dict

    Attributes
    ----------
    ids : list
        residue ids
    file: str
        pdb file
    dssp: dict
        dssp dict
    dssp_dict_keys: dict
        dssp dict keys
    """

    def __init__(self, ids, dssp, dssp_dict_keys):
        self.ids = ids
        #print(ids)
        self.dssp = dssp
        #print(dssp_dict_keys)
        self.dssp_dict_keys = dssp_dict_keys

    def get_ids(self):
        return self.ids

    def size(self):
        absolute_asa = {"S":155,"Q":225,"T":172,"Y":263,"N":195,"C":104,"K":236,"H":224,"R":274,"D":193, "E":223}
        #return sum(self.dssp[i[-2:]][2] for i in self.ids)#edited
        return sum(self.dssp[i[-2:]][3] * absolute_asa[self.dssp[i[-2:]][1]] for i in self.ids)
    def residues(self):
        #return [self.dssp[i[-2:]][0] for i in self.ids]
        return [self.dssp[i[-2:]][1] for i in self.ids]
    def patch_length(self):
        return len(self.ids)

    def residue_on_surface(self):
        """
        Get all residues accessible from the surface

        Return
        ------
        list
            residue ids
        """
        residue_list = []
        for x in self.dssp_dict_keys:
            if x[1][0] != ' ' or  x[1][2] != ' ':
                continue
            if self.dssp[x][2] > 0:
                residue_list.append(x)
        #print(residue_list)
        return residue_list
        
class ProteinPatch():

    def __init__(self, id, file, residues_in_patch, r=1.25, msms='msms -density 1.5'):
        if file.endswith('.pdb'):
            parser = PDBParser()    
        elif file.endswith('.cif'):
            parser = MMCIFParser()
        structure = parser.get_structure(id, file)
        self.model = structure[0]
        self.r = r
        if not msms.startswith('msms'):
            msms = 'msms ' + msms
        self.msms = msms
        self.residues_in_patch = residues_in_patch
        self.dssp_dict = DSSP(self.model, file)
        
        self.dssp_dict_keys = self.dssp_dict.keys()
        #print(self.dssp_dict[self.dssp_dict_keys[2]][3])#dict[key][3]
        #self.dssp_dict, self.dssp_dict_keys =  dssp_dict_from_pdb_file(file)
        self.G = self.dot_cloud_graph()
        self.G = self.patch_network(self.G)
        self.patches = self.create_patches()

    def dot_cloud_graph(self):
        """
        Create a dotted cloud of the proteins surface and label the dots
        hydrophobic or hydrophilic
        ...

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic or not
        """
        surface_points = get_surface(self.model, MSMS=self.msms)
        residue_list = [r for r in Selection.unfold_entities(self.model, "R") if seq1(r.get_resname()) != 'X']
        center_vectices = [self._sidechain_center(r.get_atoms()) for r in residue_list]

        T = KDTree(center_vectices)
		
        closest_residues = T.query(surface_points, k=1)[1]

        G = nx.Graph()
        for node, coordinates in enumerate(surface_points):
            G.add_node(node)
            G.nodes[node]['selected'] = 0
            closest_residue = residue_list[closest_residues[node]]
            if seq1(closest_residue.get_resname()) in self.residues_in_patch:
                G.nodes[node]['selected'] = 1
            G.nodes[node]['surface_vector_pos'] = coordinates
            G.nodes[node]['closest_residue_id'] = closest_residue.get_full_id()
            G.nodes[node]['closest_residue_aa'] = seq1(closest_residue.get_resname())
        return G

    def patch_network(self, G):
        """
        Create a edges between hydrophobic nodes if they are within r
        hydrophobic or hydrophilic
        ...

        Attributes
        ----------
        G : networkx.Graph
            Graph with nodes labeled as hydrophobic or not

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic and edges between nodes within range r
        """
        node_list = [i for i in G.nodes if G.nodes[i]['selected']]
        x = [G.nodes[i]['surface_vector_pos'] for i in G.nodes if G.nodes[i]['selected']]
        T = KDTree(x)
        pairs = T.query_pairs(self.r)
        G.add_edges_from([(node_list[x[0]],node_list[x[1]]) for x in pairs])
        return G

    def largest_patch(self):
        """
        get largest patch

        Return
        ------
        networkx.Graph
            Graph of the largest patch
        """
        largest_patch = max(self.patches, key=(lambda x: x.size()))
        
        return largest_patch

    def create_patches(self):
        """
        Get the components of the graph

        Return
        ------
        list
            list of Graph components where an item is a patch
        """
        patched_G = self.patch_network(self.G)
        components = [patched_G.subgraph(c) for c in nx.connected_components(patched_G)] #nx.connected_component_subgraphs(patched_G)
        patch_dict = []
        for component in components:
            if len(component.nodes) <= 1:
                continue
            residue_ids_in_patch = list(set([component.nodes[i]['closest_residue_id'] for i in component.nodes]))
            patch_dict.append(ResiduePatch(residue_ids_in_patch, self.dssp_dict, self.dssp_dict_keys))
        #print([i.size() for i in patch_dict])
        return sorted(patch_dict, key=(lambda x: x.size()), reverse=True)

    def _sidechain_center(self, atoms):
        vectors = [atom.get_vector().get_array() for atom in atoms]
        center = np.array(vectors).mean(axis=0)
        return center
def calculate_patches(infile, residues, plot, output_dir):
    """ Calculate patches on a protein surface """

    filename = os.path.basename(infile)
    outfile = filename.split('.')[0] + '.csv'
    pdb_id = filename.split('.')[0].split('_')[0]

    proteinPatches = ProteinPatch(pdb_id,infile,residues)
    patches = proteinPatches.patches
    #print([i.get_ids() for i in patches])
    result_dict = {'patch_rank':[], 'protein_id':[], 'residue_id':[], 'chain':[], 'residue_type':[], 'patch_size':[]}
    for i,patch in enumerate(patches):
        residues_in_patch = patch.get_ids()
        residues = patch.residues()
        
        for j, residue_in_patch in enumerate(residues_in_patch):
            result_dict['residue_id'].append(residue_in_patch[-2:][1][1])
            result_dict['chain'].append(residue_in_patch[-2:][0])
            result_dict['patch_size'].append(patch.size())
            result_dict['patch_rank'].append(i)
            result_dict['residue_type'].append(residues[j])
            result_dict['protein_id'].append(pdb_id)
            

    pd.DataFrame(result_dict).to_csv(os.path.join(output_dir, "patches_" + outfile), index=False)

    if plot:
        outfig = infile.split('.')[0] + '.png'
        proteinPatches.plot_largest_patches(outfig)

In [106]:
#sample
file = '../pdb_files/minus1_nocrist.pdb'
output_dir = '../output/'
residues = ["S","Q","T","Y","N","C","K","H","R","D","E"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [116]:
#sample
file = '../pdb_files/plus1.pdb'
output_dir = '../output/'
residues = ["S","Q","T","Y","N","C","K","H","R","D","E"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [122]:
#sample
file = '../pdb_files/minus2.pdb'
output_dir = '../output/'
residues = ["S","Q","T","Y","N","C","K","H","R","D","E"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [128]:
#sample
file = '../pdb_files/plus2.pdb'
output_dir = '../output/'
residues = ["S","Q","T","Y","N","C","K","H","R","D","E"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [138]:
#sample
file = '../pdb_files/plus2_ala_A.pdb'
output_dir = '../output/'
residues = ["S","Q","T","Y","N","C","K","H","R","D","E"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [147]:
for name in ['sample4','sample5','sample6','sample7','sample8','sample9']:
    file = '../pdb_files/' + name + '.pdb'
    output_dir = '../output/'
    residues = ["S","Q","T","Y","N","C","K","H","R","D","E"]
    calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

# 負電荷アミノ酸用＊N末端C末端は入れず

In [148]:
from Bio.PDB import PDBParser, MMCIFParser, Selection
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio.PDB.DSSP import DSSP
from scipy.spatial import KDTree
from Bio.SeqUtils import seq1
from pprint import pprint
import networkx as nx
import numpy as np
import pandas as pd
import os

class ResiduePatch():
    """
    Parse pisite files to dict

    Attributes
    ----------
    ids : list
        residue ids
    file: str
        pdb file
    dssp: dict
        dssp dict
    dssp_dict_keys: dict
        dssp dict keys
    """

    def __init__(self, ids, dssp, dssp_dict_keys):
        self.ids = ids
        #print(ids)
        self.dssp = dssp
        #print(dssp_dict_keys)
        self.dssp_dict_keys = dssp_dict_keys

    def get_ids(self):
        return self.ids

    def size(self):
        absolute_asa = {"D":193,"E":223}
        #return sum(self.dssp[i[-2:]][2] for i in self.ids)#edited
        return sum(self.dssp[i[-2:]][3] * absolute_asa[self.dssp[i[-2:]][1]] for i in self.ids)
    def residues(self):
        #return [self.dssp[i[-2:]][0] for i in self.ids]
        return [self.dssp[i[-2:]][1] for i in self.ids]
    def patch_length(self):
        return len(self.ids)

    def residue_on_surface(self):
        """
        Get all residues accessible from the surface

        Return
        ------
        list
            residue ids
        """
        residue_list = []
        for x in self.dssp_dict_keys:
            if x[1][0] != ' ' or  x[1][2] != ' ':
                continue
            if self.dssp[x][2] > 0:
                residue_list.append(x)
        #print(residue_list)
        return residue_list
        
class ProteinPatch():

    def __init__(self, id, file, residues_in_patch, r=1.25, msms='msms -density 1.5'):
        if file.endswith('.pdb'):
            parser = PDBParser()    
        elif file.endswith('.cif'):
            parser = MMCIFParser()
        structure = parser.get_structure(id, file)
        self.model = structure[0]
        self.r = r
        if not msms.startswith('msms'):
            msms = 'msms ' + msms
        self.msms = msms
        self.residues_in_patch = residues_in_patch
        self.dssp_dict = DSSP(self.model, file)
        
        self.dssp_dict_keys = self.dssp_dict.keys()
        #print(self.dssp_dict[self.dssp_dict_keys[2]][3])#dict[key][3]
        #self.dssp_dict, self.dssp_dict_keys =  dssp_dict_from_pdb_file(file)
        self.G = self.dot_cloud_graph()
        self.G = self.patch_network(self.G)
        self.patches = self.create_patches()

    def dot_cloud_graph(self):
        """
        Create a dotted cloud of the proteins surface and label the dots
        hydrophobic or hydrophilic
        ...

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic or not
        """
        surface_points = get_surface(self.model, MSMS=self.msms)
        residue_list = [r for r in Selection.unfold_entities(self.model, "R") if seq1(r.get_resname()) != 'X']
        center_vectices = [self._sidechain_center(r.get_atoms()) for r in residue_list]

        T = KDTree(center_vectices)
		
        closest_residues = T.query(surface_points, k=1)[1]

        G = nx.Graph()
        for node, coordinates in enumerate(surface_points):
            G.add_node(node)
            G.nodes[node]['selected'] = 0
            closest_residue = residue_list[closest_residues[node]]
            if seq1(closest_residue.get_resname()) in self.residues_in_patch:
                G.nodes[node]['selected'] = 1
            G.nodes[node]['surface_vector_pos'] = coordinates
            G.nodes[node]['closest_residue_id'] = closest_residue.get_full_id()
            G.nodes[node]['closest_residue_aa'] = seq1(closest_residue.get_resname())
        return G

    def patch_network(self, G):
        """
        Create a edges between hydrophobic nodes if they are within r
        hydrophobic or hydrophilic
        ...

        Attributes
        ----------
        G : networkx.Graph
            Graph with nodes labeled as hydrophobic or not

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic and edges between nodes within range r
        """
        node_list = [i for i in G.nodes if G.nodes[i]['selected']]
        x = [G.nodes[i]['surface_vector_pos'] for i in G.nodes if G.nodes[i]['selected']]
        T = KDTree(x)
        pairs = T.query_pairs(self.r)
        G.add_edges_from([(node_list[x[0]],node_list[x[1]]) for x in pairs])
        return G

    def largest_patch(self):
        """
        get largest patch

        Return
        ------
        networkx.Graph
            Graph of the largest patch
        """
        largest_patch = max(self.patches, key=(lambda x: x.size()))
        
        return largest_patch

    def create_patches(self):
        """
        Get the components of the graph

        Return
        ------
        list
            list of Graph components where an item is a patch
        """
        patched_G = self.patch_network(self.G)
        components = [patched_G.subgraph(c) for c in nx.connected_components(patched_G)] #nx.connected_component_subgraphs(patched_G)
        patch_dict = []
        for component in components:
            if len(component.nodes) <= 1:
                continue
            residue_ids_in_patch = list(set([component.nodes[i]['closest_residue_id'] for i in component.nodes]))
            patch_dict.append(ResiduePatch(residue_ids_in_patch, self.dssp_dict, self.dssp_dict_keys))
        #print([i.size() for i in patch_dict])
        return sorted(patch_dict, key=(lambda x: x.size()), reverse=True)

    def _sidechain_center(self, atoms):
        vectors = [atom.get_vector().get_array() for atom in atoms]
        center = np.array(vectors).mean(axis=0)
        return center
def calculate_patches(infile, residues, plot, output_dir):
    """ Calculate patches on a protein surface """

    filename = os.path.basename(infile)
    outfile = filename.split('.')[0] + '.csv'
    pdb_id = filename.split('.')[0].split('_')[0]

    proteinPatches = ProteinPatch(pdb_id,infile,residues)
    patches = proteinPatches.patches
    #print([i.get_ids() for i in patches])
    result_dict = {'patch_rank':[], 'protein_id':[], 'residue_id':[], 'chain':[], 'residue_type':[], 'patch_size':[]}
    for i,patch in enumerate(patches):
        residues_in_patch = patch.get_ids()
        residues = patch.residues()
        
        for j, residue_in_patch in enumerate(residues_in_patch):
            result_dict['residue_id'].append(residue_in_patch[-2:][1][1])
            result_dict['chain'].append(residue_in_patch[-2:][0])
            result_dict['patch_size'].append(patch.size())
            result_dict['patch_rank'].append(i)
            result_dict['residue_type'].append(residues[j])
            result_dict['protein_id'].append(pdb_id)
            

    pd.DataFrame(result_dict).to_csv(os.path.join(output_dir, "patches_" + outfile), index=False)

    if plot:
        outfig = infile.split('.')[0] + '.png'
        proteinPatches.plot_largest_patches(outfig)

In [111]:
#sample
file = '../pdb_files/minus1_nocrist.pdb'
output_dir = '../output/'
residues = ["D","E"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [124]:
#sample
file = '../pdb_files/minus2.pdb'
output_dir = '../output/'
residues = ["D","E"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [149]:
for name in ['sample4','sample5','sample6','sample7','sample8','sample9']:
    file = '../pdb_files/' + name + '.pdb'
    output_dir = '../output/'
    residues = ["D","E"]
    calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

# 正電荷アミノ酸用

In [150]:
from Bio.PDB import PDBParser, MMCIFParser, Selection
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio.PDB.DSSP import DSSP
from scipy.spatial import KDTree
from Bio.SeqUtils import seq1
from pprint import pprint
import networkx as nx
import numpy as np
import pandas as pd
import os

class ResiduePatch():
    """
    Parse pisite files to dict

    Attributes
    ----------
    ids : list
        residue ids
    file: str
        pdb file
    dssp: dict
        dssp dict
    dssp_dict_keys: dict
        dssp dict keys
    """

    def __init__(self, ids, dssp, dssp_dict_keys):
        self.ids = ids
        #print(ids)
        self.dssp = dssp
        #print(dssp_dict_keys)
        self.dssp_dict_keys = dssp_dict_keys

    def get_ids(self):
        return self.ids

    def size(self):
        absolute_asa = {"R":274,"K":236}
        #return sum(self.dssp[i[-2:]][2] for i in self.ids)#edited
        return sum(self.dssp[i[-2:]][3] * absolute_asa[self.dssp[i[-2:]][1]] for i in self.ids)
    def residues(self):
        #return [self.dssp[i[-2:]][0] for i in self.ids]
        return [self.dssp[i[-2:]][1] for i in self.ids]
    def patch_length(self):
        return len(self.ids)

    def residue_on_surface(self):
        """
        Get all residues accessible from the surface

        Return
        ------
        list
            residue ids
        """
        residue_list = []
        for x in self.dssp_dict_keys:
            if x[1][0] != ' ' or  x[1][2] != ' ':
                continue
            if self.dssp[x][2] > 0:
                residue_list.append(x)
        #print(residue_list)
        return residue_list
        
class ProteinPatch():

    def __init__(self, id, file, residues_in_patch, r=1.25, msms='msms -density 1.5'):
        if file.endswith('.pdb'):
            parser = PDBParser()    
        elif file.endswith('.cif'):
            parser = MMCIFParser()
        structure = parser.get_structure(id, file)
        self.model = structure[0]
        self.r = r
        if not msms.startswith('msms'):
            msms = 'msms ' + msms
        self.msms = msms
        self.residues_in_patch = residues_in_patch
        self.dssp_dict = DSSP(self.model, file)
        
        self.dssp_dict_keys = self.dssp_dict.keys()
        #print(self.dssp_dict[self.dssp_dict_keys[2]][3])#dict[key][3]
        #self.dssp_dict, self.dssp_dict_keys =  dssp_dict_from_pdb_file(file)
        self.G = self.dot_cloud_graph()
        self.G = self.patch_network(self.G)
        self.patches = self.create_patches()

    def dot_cloud_graph(self):
        """
        Create a dotted cloud of the proteins surface and label the dots
        hydrophobic or hydrophilic
        ...

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic or not
        """
        surface_points = get_surface(self.model, MSMS=self.msms)
        residue_list = [r for r in Selection.unfold_entities(self.model, "R") if seq1(r.get_resname()) != 'X']
        center_vectices = [self._sidechain_center(r.get_atoms()) for r in residue_list]

        T = KDTree(center_vectices)
		
        closest_residues = T.query(surface_points, k=1)[1]

        G = nx.Graph()
        for node, coordinates in enumerate(surface_points):
            G.add_node(node)
            G.nodes[node]['selected'] = 0
            closest_residue = residue_list[closest_residues[node]]
            if seq1(closest_residue.get_resname()) in self.residues_in_patch:
                G.nodes[node]['selected'] = 1
            G.nodes[node]['surface_vector_pos'] = coordinates
            G.nodes[node]['closest_residue_id'] = closest_residue.get_full_id()
            G.nodes[node]['closest_residue_aa'] = seq1(closest_residue.get_resname())
        return G

    def patch_network(self, G):
        """
        Create a edges between hydrophobic nodes if they are within r
        hydrophobic or hydrophilic
        ...

        Attributes
        ----------
        G : networkx.Graph
            Graph with nodes labeled as hydrophobic or not

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic and edges between nodes within range r
        """
        node_list = [i for i in G.nodes if G.nodes[i]['selected']]
        x = [G.nodes[i]['surface_vector_pos'] for i in G.nodes if G.nodes[i]['selected']]
        T = KDTree(x)
        pairs = T.query_pairs(self.r)
        G.add_edges_from([(node_list[x[0]],node_list[x[1]]) for x in pairs])
        return G

    def largest_patch(self):
        """
        get largest patch

        Return
        ------
        networkx.Graph
            Graph of the largest patch
        """
        largest_patch = max(self.patches, key=(lambda x: x.size()))
        
        return largest_patch

    def create_patches(self):
        """
        Get the components of the graph

        Return
        ------
        list
            list of Graph components where an item is a patch
        """
        patched_G = self.patch_network(self.G)
        components = [patched_G.subgraph(c) for c in nx.connected_components(patched_G)] #nx.connected_component_subgraphs(patched_G)
        patch_dict = []
        for component in components:
            if len(component.nodes) <= 1:
                continue
            residue_ids_in_patch = list(set([component.nodes[i]['closest_residue_id'] for i in component.nodes]))
            patch_dict.append(ResiduePatch(residue_ids_in_patch, self.dssp_dict, self.dssp_dict_keys))
        #print([i.size() for i in patch_dict])
        return sorted(patch_dict, key=(lambda x: x.size()), reverse=True)

    def _sidechain_center(self, atoms):
        vectors = [atom.get_vector().get_array() for atom in atoms]
        center = np.array(vectors).mean(axis=0)
        return center
def calculate_patches(infile, residues, plot, output_dir):
    """ Calculate patches on a protein surface """

    filename = os.path.basename(infile)
    outfile = filename.split('.')[0] + '.csv'
    pdb_id = filename.split('.')[0].split('_')[0]

    proteinPatches = ProteinPatch(pdb_id,infile,residues)
    patches = proteinPatches.patches
    #print([i.get_ids() for i in patches])
    result_dict = {'patch_rank':[], 'protein_id':[], 'residue_id':[], 'chain':[], 'residue_type':[], 'patch_size':[]}
    for i,patch in enumerate(patches):
        residues_in_patch = patch.get_ids()
        residues = patch.residues()
        
        for j, residue_in_patch in enumerate(residues_in_patch):
            result_dict['residue_id'].append(residue_in_patch[-2:][1][1])
            result_dict['chain'].append(residue_in_patch[-2:][0])
            result_dict['patch_size'].append(patch.size())
            result_dict['patch_rank'].append(i)
            result_dict['residue_type'].append(residues[j])
            result_dict['protein_id'].append(pdb_id)
            

    pd.DataFrame(result_dict).to_csv(os.path.join(output_dir, "patches_" + outfile), index=False)

    if plot:
        outfig = infile.split('.')[0] + '.png'
        proteinPatches.plot_largest_patches(outfig)

In [118]:
#sample
file = '../pdb_files/plus1.pdb'
output_dir = '../output/'
residues = ["R","K"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [130]:
#sample
file = '../pdb_files/plus2.pdb'
output_dir = '../output/'
residues = ["R","K"]
calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

In [151]:
for name in ['sample4','sample5','sample6','sample7','sample8','sample9']:
    file = '../pdb_files/' + name + '.pdb'
    output_dir = '../output/'
    residues = ["R","K"]
    calculate_patches(infile=file, residues=residues, plot=False, output_dir=output_dir)

# 特殊サンプル用

In [176]:
from Bio.PDB import PDBParser, MMCIFParser, Selection
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio.PDB.DSSP import DSSP
from scipy.spatial import KDTree
from Bio.SeqUtils import seq1
from pprint import pprint
import networkx as nx
import numpy as np
import pandas as pd
import os

class ResiduePatch():
    """
    Parse pisite files to dict

    Attributes
    ----------
    ids : list
        residue ids
    file: str
        pdb file
    dssp: dict
        dssp dict
    dssp_dict_keys: dict
        dssp dict keys
    """

    def __init__(self, ids, dssp, dssp_dict_keys):
        self.ids = ids
        #print(ids)
        self.dssp = dssp
        #print(dssp_dict_keys)
        self.dssp_dict_keys = dssp_dict_keys

    def get_ids(self):
        return self.ids

    def size(self):
        absolute_asa = {"R":274,"K":236}
        #return sum(self.dssp[i[-2:]][2] for i in self.ids)#edited
        return sum(self.dssp[i[-2:]][3] * absolute_asa[self.dssp[i[-2:]][1]] for i in self.ids)
    def residues(self):
        #return [self.dssp[i[-2:]][0] for i in self.ids]
        return [self.dssp[i[-2:]][1] for i in self.ids]
    def patch_length(self):
        return len(self.ids)

    def residue_on_surface(self):
        """
        Get all residues accessible from the surface

        Return
        ------
        list
            residue ids
        """
        residue_list = []
        for x in self.dssp_dict_keys:
            if x[1][0] != ' ' or  x[1][2] != ' ':
                continue
            if self.dssp[x][2] > 0:
                residue_list.append(x)
        #print(residue_list)
        return residue_list
        
class ProteinPatch():

    def __init__(self, id, file, residues_in_patch, residue_id, r=1.25, msms='msms -density 1.5'):
        if file.endswith('.pdb'):
            parser = PDBParser()    
        elif file.endswith('.cif'):
            parser = MMCIFParser()
        structure = parser.get_structure(id, file)
        self.model = structure[0]
        self.r = r
        if not msms.startswith('msms'):
            msms = 'msms ' + msms
        self.msms = msms
        self.residues_in_patch = residues_in_patch
        self.residue_id = residue_id
        self.dssp_dict = DSSP(self.model, file)
        
        self.dssp_dict_keys = self.dssp_dict.keys()
        #print(self.dssp_dict[self.dssp_dict_keys[2]][3])#dict[key][3]
        #self.dssp_dict, self.dssp_dict_keys =  dssp_dict_from_pdb_file(file)
        self.G = self.dot_cloud_graph()
        self.G = self.patch_network(self.G)
        self.patches = self.create_patches()

    def dot_cloud_graph(self):
        """
        Create a dotted cloud of the proteins surface and label the dots
        hydrophobic or hydrophilic
        ...

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic or not
        """
        surface_points = get_surface(self.model, MSMS=self.msms)
        residue_list = [r for r in Selection.unfold_entities(self.model, "R") if seq1(r.get_resname()) != 'X']
        center_vectices = [self._sidechain_center(r.get_atoms()) for r in residue_list]

        T = KDTree(center_vectices)
		
        closest_residues = T.query(surface_points, k=1)[1]

        G = nx.Graph()
        for node, coordinates in enumerate(surface_points):
            G.add_node(node)
            G.nodes[node]['selected'] = 0
            closest_residue = residue_list[closest_residues[node]]
            if closest_residue.get_id()[1] in self.residue_id:
                G.nodes[node]['selected'] = 1
            G.nodes[node]['surface_vector_pos'] = coordinates
            G.nodes[node]['closest_residue_id'] = closest_residue.get_full_id()
            G.nodes[node]['closest_residue_aa'] = seq1(closest_residue.get_resname())
        return G

    def patch_network(self, G):
        """
        Create a edges between hydrophobic nodes if they are within r
        hydrophobic or hydrophilic
        ...

        Attributes
        ----------
        G : networkx.Graph
            Graph with nodes labeled as hydrophobic or not

        Return
        ------
        networkx.Graph
            Graph with nodes labeled as hydrophobic and edges between nodes within range r
        """
        node_list = [i for i in G.nodes if G.nodes[i]['selected']]
        x = [G.nodes[i]['surface_vector_pos'] for i in G.nodes if G.nodes[i]['selected']]
        T = KDTree(x)
        pairs = T.query_pairs(self.r)
        G.add_edges_from([(node_list[x[0]],node_list[x[1]]) for x in pairs])
        return G

    def largest_patch(self):
        """
        get largest patch

        Return
        ------
        networkx.Graph
            Graph of the largest patch
        """
        largest_patch = max(self.patches, key=(lambda x: x.size()))
        
        return largest_patch

    def create_patches(self):
        """
        Get the components of the graph

        Return
        ------
        list
            list of Graph components where an item is a patch
        """
        patched_G = self.patch_network(self.G)
        components = [patched_G.subgraph(c) for c in nx.connected_components(patched_G)] #nx.connected_component_subgraphs(patched_G)
        patch_dict = []
        for component in components:
            if len(component.nodes) <= 1:
                continue
            residue_ids_in_patch = list(set([component.nodes[i]['closest_residue_id'] for i in component.nodes]))
            patch_dict.append(ResiduePatch(residue_ids_in_patch, self.dssp_dict, self.dssp_dict_keys))
        #print([i.size() for i in patch_dict])
        return sorted(patch_dict, key=(lambda x: x.size()), reverse=True)

    def _sidechain_center(self, atoms):
        vectors = [atom.get_vector().get_array() for atom in atoms]
        center = np.array(vectors).mean(axis=0)
        return center
def calculate_patches(infile, residues, residue_id, plot, output_dir):
    """ Calculate patches on a protein surface """

    filename = os.path.basename(infile)
    outfile = filename.split('.')[0] + '.csv'
    pdb_id = filename.split('.')[0].split('_')[0]

    proteinPatches = ProteinPatch(pdb_id,infile,residues, residue_id)
    patches = proteinPatches.patches
    #print([i.get_ids() for i in patches])
    result_dict = {'patch_rank':[], 'protein_id':[], 'residue_id':[], 'chain':[], 'residue_type':[], 'patch_size':[]}
    for i,patch in enumerate(patches):
        residues_in_patch = patch.get_ids()
        residues = patch.residues()
        
        for j, residue_in_patch in enumerate(residues_in_patch):
            result_dict['residue_id'].append(residue_in_patch[-2:][1][1])
            result_dict['chain'].append(residue_in_patch[-2:][0])
            result_dict['patch_size'].append(patch.size())
            result_dict['patch_rank'].append(i)
            result_dict['residue_type'].append(residues[j])
            result_dict['protein_id'].append(pdb_id)
            

    pd.DataFrame(result_dict).to_csv(os.path.join(output_dir, "patches_" + outfile), index=False)

    if plot:
        outfig = infile.split('.')[0] + '.png'
        proteinPatches.plot_largest_patches(outfig)

In [187]:
#sample
file = '../pdb_files/plus2_ala_A.pdb'
output_dir = '../output/'
residues = ["R","K"]
residue_id = [3,50]
calculate_patches(infile=file, residues=residues, residue_id=residue_id, plot=False, output_dir=output_dir)

In [46]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

# PDBファイルのパス
pdb_file = './minus1_nocrist.pdb'
structure_id = 'minus1_nocrist'

# PDBParserを使用してPDBファイルを解析
parser = PDBParser()
structure = parser.get_structure(structure_id, pdb_file)

# DSSPを使用して構造の2次構造を取得
model = structure[0]

# DSSPの実行ファイルのパス
dssp_exe = '/usr/bin/dssp'

try:
    dssp = DSSP(model, pdb_file, dssp=dssp_exe)
    dssp_dict = dict(dssp)
    print(dssp_dict)
except Exception as e:
    print(f"Error: {e}")

{('A', (' ', 1, ' ')): (1, 'A', '-', 1.0, 360.0, 153.5, 0, 0.0, 54, -0.1, 0, 0.0, 2, -0.1), ('A', (' ', 2, ' ')): (2, 'P', '-', 0.3897058823529412, -72.2, 153.0, 0, 0.0, 3, -0.7, 0, 0.0, 4, -0.5), ('A', (' ', 3, ' ')): (3, 'D', 'G', 0.6748466257668712, -66.5, -33.3, 1, -0.2, 3, -1.4, 2, -0.2, 4, -0.7), ('A', (' ', 4, ' ')): (4, 'F', 'G', 0.22842639593908629, -64.1, -28.8, 1, -0.3, 3, -0.7, 2, -0.2, -1, -0.2), ('A', (' ', 5, ' ')): (5, 'C', 'G', 0.022222222222222223, -62.6, -31.1, -3, -0.7, 20, -0.4, 1, -0.2, -1, -0.3), ('A', (' ', 6, ' ')): (6, 'L', 'G', 0.7621951219512195, -69.7, -30.5, -3, -1.4, -1, -0.2, -4, -0.5, -2, -0.2), ('A', (' ', 7, ' ')): (7, 'E', 'S', 0.5360824742268041, -75.3, 155.2, -4, -0.7, 18, -0.1, -3, -0.7, 16, -0.1), ('A', (' ', 8, ' ')): (8, 'P', 'P', 0.7573529411764706, -71.6, 160.0, 0, 0.0, 35, -0.2, 0, 0.0, -1, -0.1), ('A', (' ', 9, ' ')): (9, 'P', 'P', 0.3382352941176471, -64.8, 144.6, 0, 0.0, 2, -0.6, 0, 0.0, 35, -0.1), ('A', (' ', 10, ' ')): (10, 'Y', '-', 0.